# 와인데이터 크롤링해오기 

In [66]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import signal
from tqdm import trange, notebook  
import time
import math

In [32]:
driver = webdriver.Chrome('./driver/chromedriver')

In [33]:
url = 'https://www.wine21.com/13_search/wine_list.html'
driver.get(url)

In [35]:
# 평점순으로 (변경되기까지 시간이 오래걸림)
xpath = '//*[@id="shOrder1"]/option[4]'
driver.find_element(By.XPATH,xpath).click()

- 첫번째 게시글 클릭 후 
- 상품명, 도수, 생산국가/지역, 품종, 추천음식, 음용온도  (상세정보 크롤링)

In [36]:
# 첫번째 와인 클릭 
driver.find_element(By.XPATH,'//*[@id="wine_list"]/li[1]/div[2]/div[1]/h3/a').click()
time.sleep(1)

In [19]:
# 상세정보 가져오기 
wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]').text
wine_name = driver.find_element(By.CLASS_NAME,'wine-name').text
wine_flavor = driver.find_element(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[2]').text
# flavor a 태그 class="on"으로 들어가있기때문에 값이 나오지않음 ..
wine_food = driver.find_elements(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[3]/div[2]/div[2]/ul/li/p')
# wind_food가 리스트로 들어감 -> 리스트 안에 리스트가 있어버림 
# 데이터프레임으로 만들때 중첩리스트!! 기억할 것 
wine_price = driver.find_element(By.CSS_SELECTOR,'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong').text
wine_text = driver.find_element(By.CLASS_NAME,'view-tab-inner').text
wine = [wine_category,wine_name,wine_flavor, wine_food, wine_capacity,wine_text]

# 전체 목록으로 돌아가기 
driver.back() 

In [20]:
wine

['레드',
 "7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",
 '당도\n산도\n바디\n타닌',
 [<selenium.webdriver.remote.webelement.WebElement (session="57b447aaba429c90e7978973f7bb1121", element="70EA32B4C633FD5292C4A377FD29B9BB_element_667")>,
  <selenium.webdriver.remote.webelement.WebElement (session="57b447aaba429c90e7978973f7bb1121", element="70EA32B4C633FD5292C4A377FD29B9BB_element_668")>,
  <selenium.webdriver.remote.webelement.WebElement (session="57b447aaba429c90e7978973f7bb1121", element="70EA32B4C633FD5292C4A377FD29B9BB_element_669")>],
 '36,000원',
 '생산자\n세븐 컬러즈 7 Colores\n국가/생산지역\n칠레(Chile) > 센트럴 밸리(Central Valley)\n주요품종\n카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat) 5%\n스타일\nChilean Cabernet Sauvignon\n음용온도\n16~18 ℃\n추천음식\n채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.\n수입사\n씨에스알와인   ']

In [24]:
driver.find_element(By.CSS_SELECTOR,'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong').text
# 용량만 가져오고 싶은데 가격도 가져와짐 

'36,000원'

In [95]:
driver.find_element(By.XPATH,'/html/body/section/div[3]/div[2]/div[2]/div[2]').text

'당도\n산도\n바디\n타닌'

In [ ]:
test = []

In [60]:
test = pd.DataFrame()
for i in range(1,3):
    try : 
        # 와인 클릭 
        driver.find_element(By.XPATH,'//*[@id="wine_list"]/li['+str(i)+']/div[2]/div[1]/h3/a').click()
        time.sleep(1)
        
        # 상세정보 가져오기 
        wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]')
        wine_name = driver.find_element(By.CLASS_NAME,'wine-name')
        wine_price = driver.find_element(By.CSS_SELECTOR,'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong')
        wine_text = driver.find_element(By.CLASS_NAME,'view-tab-inner')

        # 전체 목록으로 돌아가기 
        driver.back() 
        
        res_category = [c.text for c in wine_category]
        res_name = [n.text for n in wine_name]
        res_price = [p.text for p in wine_price]
        res_text = [t.text for t in wine_text]
        
        wine = [res_category, res_name, res_price, res_text]
        wines.append(wine)
        
        # 데이터프레임에 저장 
#         test = test.append(pd.DataFrame({'중분류':res_category, '상품명':res_name, '가격':res_price,'특징':res_text})) 
        test = pd.DataFrame(wines, columns = ['중분류', '상품명','가격','특징'])     
    except :
        pass

TypeError: 'WebElement' object is not iterable

- 500개의 데이터만 가져와서 데이터프레임에 넣기 

###### 다시 진행 

In [44]:
result=[]
result = pd.DataFrame()
# 가져올 순서 -> 중분류, 상품명, 도수, 생산국가, 생산자, 주요품종, 음용온도, 추천음식, 맛평가
# 용량은 가져올 수 없음 

wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]').text
# wine_name = driver.find_element(By.CLASS_NAME,'wine-name').text
wine_origin = driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[2]').text
# 
wine_detail = driver.find_element(By.XPATH, '//*[@id="detail"]/div/div').text

# res_wine = [w.text for w in wine]
result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=[0])) 

# # 전체 목록으로 돌아가기 
# driver.back() 

C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\4104372711.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=[0]))


In [45]:
result

,중분류,상품명,생산국가,상세정보
0,화이트,갈로 리빙스톤 콩코드,호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...


In [74]:
# 홈페이지 연결
url = 'https://www.wine21.com/13_search/wine_list.html'
driver.get(url)
time.sleep(2)

# 평점순
xpath = '//*[@id="shOrder1"]/option[4]'
driver.find_element(By.XPATH,xpath).click()
time.sleep(15)

In [71]:
# 와인 페이지 더보기 버튼 끝까지 클릭
# 28,474개의 와인으로 시간이 오래걸림 
while driver.find_element(By.CSS_SELECTOR, '#wineListMoreBtn').is_displayed:
    try:
        WebDriverWait(driver,1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#wineListMoreBtn"))
        ).click()
    except TimeoutException:
        break
# 450개까지 나오고 진행불가 (노트북 문제이ㅣㄹ수도 있음!  )

NameError: name 'TimeoutException' is not defined

In [77]:
result=[]
result = pd.DataFrame()
# 와인 50개의 데이터 가져오기 
for i in notebook.tqdm(range(0,51)):
    try : 
        # 와인 클릭 
        driver.find_element(By.XPATH,'//*[@id="wine_list"]/li['+str(i)+']/div[2]/div[1]/h3/a').click()
        time.sleep(3)
        
        # 가져올 순서 -> 중분류, 상품명, 생산국가, 상세정보
       
        wine_category=driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[1]').text
        wine_name = driver.find_element(By.CLASS_NAME,'wine-name').text
        wine_origin = driver.find_element(By.XPATH, '/html/body/section/div[3]/div[2]/div[2]/div[1]/p/span[2]').text

        wine_detail = driver.find_element(By.XPATH, '//*[@id="detail"]/div/div').text
        
        # 전체 목록으로 돌아가기 
        driver.back()        
        time.sleep(8)
        
        # 데이터프레임에 저장
        result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine'])) 

    except :
        pass

  0%|          | 0/111 [00:00<?, ?it/s]

C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\U

C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\Users\guddk\AppData\Local\Temp\ipykernel_22824\752813769.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(pd.DataFrame({'중분류':wine_category, '상품명':wine_name,'생산국가':wine_origin,'상세정보':wine_detail},index=['wine']))
C:\U

In [78]:
result # 50개 

,중분류,상품명,생산국가,상세정보
wine,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,생산자\n세븐 컬러즈 7 Colores\n국가/생산지역\n칠레(Chile) > 센트...
wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...
wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...
wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...
wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...
wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...
wine,화이트,"까브 드 루그니, 마콩 루그니 유진 블랑 샤르도네",프랑스,생산자\n까브 드 루그니 Cave de Lugny\n국가/생산지역\n프랑스(Fran...
wine,화이트,"깐티네 포베로, 모스카토 다스티 '캄포 델 팔리오'",이탈리아,생산자\n깐띠네 포베로 Cantine Povero\n국가/생산지역\n이탈리아(Ita...
wine,레드,꼬모 까베르네 소비뇽,칠레,생산자\n운두라가 Undurraga\n국가/생산지역\n칠레(Chile) > 센트럴 ...
wine,스파클링,노비볼레 로마냐 스푸만테 비앙코,이탈리아,생산자\n까비로 Caviro sca-Forli\n국가/생산지역\n이탈리아(Italy...


In [79]:
# csv 파일로 저장하기 
result.to_csv('./wine50_list.csv', encoding = 'utf-8-sig')